In [ ]:
import pandas as pd
import numpy as np
import os

data = pd.read_excel(r'C:\Users\Qing Ye\Code-And-Note\Business\Data-Engineering\Scripts\Temerature Maping Table.xlsx', sheet_name='Sheet1')
df = pd.DataFrame(data)

print(df)

In [ ]:
import urllib
from sqlalchemy import create_engine, inspect

DATABASES = {
  'default': {
     'ENGINE': 'mssql',
     'NAME': "miebachcapacitytooldb",
     'USER': 'miebachindianapolis',
     "PASSWORD": 'Miebach1',
     "HOST": 'miebachinternalprojectdbs.database.windows.net',
     "PORT": "1433",
     "OPTIONS": {"driver": "ODBC Driver 17 for SQL Server",
             },
     }
}

# Admin_Tool_Global
# miebachinternal

def connect_database():

    driver = DATABASES['default']['OPTIONS']['driver']
    server = DATABASES['default']['HOST']
    database = DATABASES['default']['NAME']
    username = DATABASES['default']['USER']
    password = DATABASES['default']['PASSWORD']

    # connection parameters
    params = 'DRIVER=' + driver + ';' \
        'SERVER=' + server + ';' \
        'DATABASE=' + database + ';' \
        'UID=' + username + ';' \
        'PWD=' + password + ';'

    conn_params = urllib.parse.quote_plus(params)
    dbconn = create_engine('mssql+pyodbc:///?odbc_connect=%s' %
                           conn_params, fast_executemany=False)

    autocommit_engine = dbconn.execution_options(isolation_level="AUTOCOMMIT")
    return dbconn
engine = connect_database()

# ------------------------------------------------------------------------------------

# with engine.begin() as connection:
#     for index, row in df.iterrows():
#         update_query = f"""
#         UPDATE core_user
#         SET working_office_id = {row['working_office_id']},
#             position_id = {row['position_id']},
#             is_active = {row['is_active']},
#             pillar_item_id = {row['pillar_id']}
#         WHERE id = {row['id']}
#         """
#         connection.execute(update_query)
# print('DONE')

# ------------------------------------------------------------------------------------

# datatable='baseline_systemarea'

# df.to_sql(datatable, engine, if_exists='append', index=False)
# print('Done')

# ------------------------------------------------------------------------------------

datatable = 'baseline_matchtemperature'

# Iterate through the DataFrame
for index, row in df.iterrows():
    given_temp = row['given_temp']
    
    # Check if the given_temp value already exists in the table
    query = f"SELECT COUNT(*) FROM {datatable} WHERE given_temp = ?"
    result = engine.execute(query, (given_temp,)).scalar()
    
    if result == 0:
        # If the value does not exist, insert the row
        row.to_frame().T.to_sql(datatable, engine, if_exists='append', index=False)
        print(f'Inserted row with given_temp = {given_temp}')
    else:
        print(f'Skipped row with given_temp = {given_temp}, already exists.')

print('Done')